In [ ]:
# import os
# import pandas as pd

# BASE_DIR = 'data/'
# train_folder = BASE_DIR+'train data/Tea leaf blight'
# # train_annotation = BASE_DIR+'annotated_train_data/'

# files_in_train = sorted(os.listdir(train_folder))
# # files_in_annotated = sorted(os.listdir(train_annotation))

# images=[i for i in files_in_train]

# df = pd.DataFrame()
# df['images']=[train_folder+str(x) for x in images]
# # df['labels']=[train_annotation+str(x) for x in images]

# pd.to_csv('files_path.csv', header=None)

In [1]:
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import argparse
 
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required = True, help = "Path to the image")
# args = vars(ap.parse_args())
 
# load the image and convert it to a floating point data type

image = img_as_float(io.imread("1.jpg")) 
# loop over the number of segments
for numSegments in (100, 200, 300):
	# apply SLIC and extract (approximately) the supplied number
	# of segments
	segments = slic(image, n_segments = numSegments, sigma = 5)
 
	# show the output of SLIC
	fig = plt.figure("Superpixels -- %d segments" % (numSegments))
	ax = fig.add_subplot(1, 1, 1)
	ax.imshow(mark_boundaries(image, segments))
	plt.axis("off")
   
# show the plots
plt.show()
fig.savefig('plot.png')  

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [2]:
from cv2 import cv2
import numpy as np

filename = 'plot.png'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# find Harris corners
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
dst = cv2.dilate(dst,None)
ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

cv2.imwrite('harris.png',img)

True

In [3]:
# src = cv2.imread("harris.png", 1) # read input image
# gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) # convert to grayscale
# blur = cv2.blur(gray, (3, 3)) # blur the image
# ret, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)
# contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# hull = []
 
# # calculate points for each contour
# for i in range(len(contours)):
#     # creating convex hull object for each contour
#     hull.append(cv2.convexHull(contours[i], False))

# drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
 
# # draw contours and hull points
# for i in range(len(contours)):
#     color_contours = (0, 255, 0) # green - color for contours
#     color = (255, 0, 0) # blue - color for convex hull
#     # draw ith contour
#     cv2.drawContours(drawing, contours, i, color_contours, 1, 8, hierarchy)
#     # draw ith convex hull object
#     cv2.drawContours(drawing, hull, i, color, 1, 8) 
# cv2.imshow('Contours', drawing)     

In [4]:
import random as rng
rng.seed(12345)

def thresh_callback(val):
    threshold = val
    # Detect edges using Canny
    canny_output = cv2.Canny(src_gray, threshold, threshold * 2)
    # Find contours
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(contours)):
        hull = cv2.convexHull(contours[i])
        hull_list.append(hull)
    # Draw contours + hull results
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.drawContours(drawing, contours, i, color)
        cv2.drawContours(drawing, hull_list, i, color)
    # Show in a window
#     cv2.imshow('Contours', drawing)
    cv2.imwrite('contours.png',drawing)

In [5]:
src = cv2.imread('harris.png',-1)
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
src_gray = cv2.blur(src_gray, (3,3))
# Create Window
source_window = 'Source'
cv2.namedWindow(source_window)
# cv2.imshow(source_window, src)
max_thresh = 255
thresh = 100 # initial threshold
cv2.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [1]:
from skimage.feature import greycomatrix,greycoprops
from skimage import feature
import numpy as np
from skimage.io import imread
from skimage import io
from sklearn import preprocessing
mat = []
for i in range(60):
    img = io.imread("test_contours/"+str(i)+".png", as_grey=True)
    S = preprocessing.MinMaxScaler((0,11)).fit_transform(img).astype(int)
    S[99:105, 99:105]
    io.imsave('preprocessed.png', S)
    img = io.imread('preprocessed.png')
    # clipped = np.clip(img, a_min=0, a_max=2)
    # result = greycomatrix(img, distances = [1], angles = [0], levels = 3, symmetric=True)
    # img = np.uint8(img)
    result = greycomatrix(img, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm = result[0:5, 0:5, 0, 0]
    mat.insert(i,glcm)
# GLCM = greycomatrix(img,'Offset',[0, np.pi/2]);
# stats = greycoprops(GLCM,'all')
# t1= struct2array(stats)

/home/polkampally/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: preprocessed.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/polkampally/anaconda3/lib/python3.6/site-packages/skimage/feature/texture.py:109: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(image.dtype, np.float):


In [13]:
feature.greycoprops(result, 'contrast')

array([[504950]])

In [2]:
mat

[array([[102543,    181,    827,    863,    580],
        [   169,    205,     33,      6,      3],
        [   797,     30,    554,     86,      9],
        [   846,     11,     80,    646,     42],
        [   595,      3,     20,     33,    398]], dtype=uint32),
 array([[102736,    288,    471,    549,    827],
        [   271,    195,     16,     13,     24],
        [   477,      9,    381,     26,     13],
        [   551,      8,     22,    381,     33],
        [   798,     23,     11,     34,    763]], dtype=uint32),
 array([[101056,    182,    863,    887,    786],
        [   189,     97,     10,      4,     16],
        [   873,      6,    768,     39,     39],
        [   830,      4,     36,    686,     45],
        [   780,     14,     47,     42,    729]], dtype=uint32),
 array([[104014,    634,    399,    735,    545],
        [   631,    636,     13,     10,     12],
        [   422,      8,    338,     21,     23],
        [   739,     19,     14,    698,     36],
  